In [2]:
from sortabilitytime.sortnregress_time import (
    var_sort_regress,
    r2_sort_regress_ts,
    r2_sort_regress,
    random_sort_regress_ts,
    random_sort_regress,
    var_sort_regress_reverse,
    var_sort_regress,
    var_sort_regress_ts,
    var_sort_regress_ts_reverse,
    ts_graph_to_summary_graph,
)
from sortabilitytime.dynotears import Dynotears
from sortabilitytime.sortability_ts import var_sortability, r2_sortability
from tigramite.pcmci import PCMCI
from tigramite.independence_tests.parcorr import ParCorr
import tigramite.data_processing as pp
from sklearn.metrics import f1_score
from causalchamber.datasets import Dataset
import tigramite.data_processing as pp
from causalchamber.ground_truth import graph
from tigramite.independence_tests.parcorr import ParCorr
from causalchamber.ground_truth import graph
# ignore warnings
import warnings
import pandas as pd
warnings.filterwarnings("ignore")
import numpy as np


Fetching list of available datasets from https://causalchamber.s3.eu-central-1.amazonaws.com/downloadables/directory.yaml ... done.
If you use our datasets or simulators for your work please consider citing:

﻿@article{gamella2025chamber,
  author={Gamella, Juan L. and Peters, Jonas and B{"u}hlmann, Peter},
  title={Causal chambers as a real-world physical testbed for {AI} methodology},
  journal={Nature Machine Intelligence},
  doi={10.1038/s42256-024-00964-x},
  year={2025},
}



In [3]:
parcorr = ParCorr(
    significance="analytic",
    #                   mask_type='y'
)

In [4]:
tau_max = 2

In [ ]:
# Provided dataset information
dataset_info = [
    {"name": "lt_camera_walks_v1", "chamber": "Light tunnel", "config": "camera"},
    {"name": "lt_color_regression_v1", "chamber": "Light tunnel", "config": "camera"},
    {
        "name": "lt_interventions_standard_v1",
        "chamber": "Light tunnel",
        "config": "standard",
    },
    {"name": "lt_walks_v1", "chamber": "Light tunnel", "config": "standard"},
    {"name": "wt_walks_v1", "chamber": "Wind tunnel", "config": "standard"},
    {"name": "lt_malus_v1", "chamber": "Light tunnel", "config": "standard"},
    {"name": "wt_bernoulli_v1", "chamber": "Wind tunnel", "config": "standard"},
    {"name": "wt_changepoints_v1", "chamber": "Wind tunnel", "config": "standard"},
    {"name": "wt_intake_impulse_v1", "chamber": "Wind tunnel", "config": "standard"},
    {
        "name": "wt_pressure_control_v1",
        "chamber": "Wind tunnel",
        "config": "pressure-control",
    },
    {"name": "lt_test_v1", "chamber": "Light tunnel", "config": "standard"},
    {"name": "wt_test_v1", "chamber": "Wind tunnel", "config": "standard"},
    {"name": "lt_camera_test_v1", "chamber": "Light tunnel", "config": "camera"},
    {"name": "wt_validate_v1", "chamber": "Wind tunnel", "config": "standard"},
    {
        "name": "wt_pc_validate_v1",
        "chamber": "Wind tunnel",
        "config": "pressure-control",
    },
    {"name": "lt_validate_v1", "chamber": "Light tunnel", "config": "standard"},
    {"name": "lt_camera_validate_v1", "chamber": "Light tunnel", "config": "standard"},
]

results_dict = {}

f1_scores = {
    "f1_var_sortnregress": [],
    "f1_r2_sortnregress": [],
    "f1_random": [],
}

for dataset_info_entry in dataset_info:
    dataset_name = dataset_info_entry["name"]
    chamber = dataset_info_entry["chamber"]
    config = dataset_info_entry["config"]
    if chamber.lower() == "wind tunnel":
        short_name = "wt"
    elif chamber.lower() == "light tunnel":
        short_name = "lt"
    dataset = Dataset(name=dataset_name, root="../Data/Causalchamber", download=True)
    ground_truth = graph(chamber=short_name, configuration=config)

    for experiment in dataset.available_experiments():
        ground_truth = graph(chamber=short_name, configuration=config)

        print(
            f"running {experiment} on {dataset_name} with config {config} and chamber {short_name}"
        )
        df = dataset.get_experiment(name=experiment).as_pandas_dataframe()

        # remove nans and infs from df
        df = df.replace([np.inf, -np.inf], np.nan)
        df = df.dropna()

        try:
            df = df[ground_truth.columns]

        except KeyError as e:
            print(
            f"error in {experiment} on {dataset_name} with config {config} and chamber {short_name}"
            )
            ground_truth = ground_truth.drop(columns=["im"])
            ground_truth = ground_truth.drop(index=["im"])
            df = df[ground_truth.columns]

        var = var_sortability(df.to_numpy(), ground_truth.to_numpy())
        r2 = r2_sortability(df.to_numpy(), ground_truth.to_numpy())

        # remove infs and nans in gruond_truth
        ground_truth = ground_truth.replace([np.inf, -np.inf], np.nan)
        ground_truth = ground_truth.dropna()

        # also in data
        df = df.replace([np.inf, -np.inf], np.nan)

        df = df.dropna()

        data = df.to_numpy()

        dataframe = pp.DataFrame(
            data.copy(), mask=None, datatime={0: np.arange(len(data))}
        )
        # dynotears = Dynotears(dataframe=dataframe)
        # w_est, a_est, string_graph, a_est_dyno = dynotears.run_dynotears(
        #     tau_max=3, max_iter=100, w_threshold=0.1, lambda_a=0.1, lambda_w=0.1
        # )

        # a_est_dyno = ts_graph_to_summary_graph(a_est_dyno)

        # f1_dynotears = f1_score(ground_truth.to_numpy().flatten(), a_est_dyno.flatten())
        try:
            data = df.to_numpy()
            dataframe = pp.DataFrame(data)
            pcmci = PCMCI(dataframe=dataframe, cond_ind_test=parcorr, verbosity=0)
            results_pcmci = pcmci.run_pcmciplus(tau_max=3, pc_alpha=0.01)
            results_pcmci["p_matrix"] = np.where(
                results_pcmci["p_matrix"] < 0.01, results_pcmci["p_matrix"], 0
            )
            a_pcmci = results_pcmci["p_matrix"]
            a_pcmci[a_pcmci != 0] = 1

            a_pcmci = ts_graph_to_summary_graph(a_pcmci)

            G_true = ground_truth.to_numpy()

            a_est_var_sortnregress = var_sort_regress_ts(data, tau_max=tau_max)

            a_est_var_sortnregress_reverse = var_sort_regress_ts_reverse(data, tau_max=tau_max)
            a_est_r2_sortnregress = r2_sort_regress_ts(data, tau_max=tau_max)
            a_est_random = random_sort_regress_ts(data, tau_max=tau_max)
            a_est_var_sortnregress[:, :, 0] = var_sort_regress(data)
            a_est_r2_sortnregress[:, :, 0] = r2_sort_regress(data)
            a_est_var_sortnregress_reverse[:, :, 0] = var_sort_regress_reverse(data)
            a_est_random[:, :, 0] = random_sort_regress(data)
            a_est_var_sortnregress = ts_graph_to_summary_graph(a_est_var_sortnregress)

            a_est_var_sortnregress_reverse = ts_graph_to_summary_graph(
                a_est_var_sortnregress_reverse
            )

            a_est_r2_sortnregress = ts_graph_to_summary_graph(a_est_r2_sortnregress)

            a_est_random = ts_graph_to_summary_graph(a_est_random)

            # replace all values above !=0 with 1
            a_est_var_sortnregress[a_est_var_sortnregress != 0] = 1
            a_est_r2_sortnregress[a_est_r2_sortnregress != 0] = 1
            a_est_random[a_est_random != 0] = 1
            f1_var_sortnregress = f1_score(
                G_true.flatten(), a_est_var_sortnregress.flatten()
            )

            f1_var_sortnregress_reverse = f1_score(
                G_true.flatten(), a_est_var_sortnregress_reverse.flatten()
            )
            f1_r2_sortnregress = f1_score(G_true.flatten(), a_est_r2_sortnregress.flatten())
            f1_random = f1_score(G_true.flatten(), a_est_random.flatten())
            f1_pcmci = f1_score(G_true.flatten(), a_pcmci.flatten())

            result = {
                "var_sortability": var,
                "r2_sortability": r2,
                # "f1_dynotears": f1_dynotears,
                "f1_pcmci": f1_pcmci,
                "f1_var_sortnregress": f1_var_sortnregress,
                "f1_var_sortnregress_reverse": f1_var_sortnregress_reverse,
                "f1_r2_sortnregress": f1_r2_sortnregress,
                "f1_random": f1_random,
            }
            results_dict[f"{dataset_name}+{experiment}"] = result

        except: 
            result = {
                "var_sortability": var,
                "r2_sortability": r2,
                # "f1_dynotears": f1_dynotears,
                "f1_pcmci": np.nan,
                "f1_var_sortnregress": np.nan,
                "f1_var_sortnregress_reverse": np.nan,
                "f1_r2_sortnregress": np.nan,
                "f1_random": np.nan,
            }
            results_dict[f"{dataset_name}+{experiment}"] = result

Dataset lt_camera_walks_v1 found in "../../Data/Causalchamber/lt_camera_walks_v1".
running color_mix on lt_camera_walks_v1 with config camera and chamber lt
error in color_mix on lt_camera_walks_v1 with config camera and chamber lt
fallback
fallback
fallback
running actuator_mix on lt_camera_walks_v1 with config camera and chamber lt
error in actuator_mix on lt_camera_walks_v1 with config camera and chamber lt
fallback
fallback
fallback
Dataset lt_color_regression_v1 found in "../../Data/Causalchamber/lt_color_regression_v1".
running pol_1_90 on lt_color_regression_v1 with config camera and chamber lt
error in pol_1_90 on lt_color_regression_v1 with config camera and chamber lt
fallback
fallback
fallback
running aperture_11.0 on lt_color_regression_v1 with config camera and chamber lt
error in aperture_11.0 on lt_color_regression_v1 with config camera and chamber lt
fallback
fallback
fallback
running iso_1000.0 on lt_color_regression_v1 with config camera and chamber lt
error in iso_10

In [3]:
df = pd.DataFrame(results_dict)

df = df.T

# remove every value with no + in index
for index in df.index:
    if "+" not in index:
        df = df.drop(index)


# round to 2 decimals
df = df.round(2)

# experiment column is after the + in index
df["Experiment"] = df.index.str.split("+").str[1]

# dataset column is before the + in index
df["Dataset"] = df.index.str.split("+").str[0]

# remove index and set dataset as first and experiment as second column
df = df.reset_index(drop=True)

# df = df[["Dataset", "Experiment", "var_sortability", "r2_sortability"]]

# df.to_latex("results_causalchamber.txt")



In [ ]:
# Define the columns to aggregate
agg_columns = [
    "var_sortability",
    "r2_sortability",
    "f1_pcmci",
    "f1_var_sortnregress",
    "f1_var_sortnregress_reverse",
    "f1_r2_sortnregress",
    "f1_random",
]
df_grouped = df.groupby("Dataset").agg({col: ["mean", "std"] for col in agg_columns})
df_grouped = df_grouped.round(2)
df_grouped = df_grouped.fillna(0)

df_grouped.columns = ["_".join(col) for col in df_grouped.columns]

for col in agg_columns:
    mean_col = f"{col}_mean"
    std_col = f"{col}_std"
    df_grouped[col] = (
        df_grouped[mean_col].astype(str) + " $\pm$ " + df_grouped[std_col].astype(str)
    )

df_grouped = df_grouped.drop(
    columns=[f"{col}_mean" for col in agg_columns]
    + [f"{col}_std" for col in agg_columns]
)
df_grouped

,var_sortability,r2_sortability,f1_pcmci,f1_var_sortnregress,f1_var_sortnregress_reverse,f1_r2_sortnregress,f1_random
Dataset,,,,,,,
lt_camera_test_v1,0.94 $\pm$ 0.01,0.25 $\pm$ 0.24,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0
lt_camera_validate_v1,0.99 $\pm$ 0.02,0.01 $\pm$ 0.03,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0
lt_camera_walks_v1,0.95 $\pm$ 0.0,0.23 $\pm$ 0.07,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0
lt_color_regression_v1,0.94 $\pm$ 0.02,0.15 $\pm$ 0.06,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0
lt_interventions_standard_v1,0.94 $\pm$ 0.03,0.46 $\pm$ 0.04,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0
lt_malus_v1,0.98 $\pm$ 0.02,0.02 $\pm$ 0.01,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0
lt_test_v1,0.96 $\pm$ 0.03,0.01 $\pm$ 0.02,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0
lt_validate_v1,0.99 $\pm$ 0.02,0.02 $\pm$ 0.02,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0
lt_walks_v1,0.9 $\pm$ 0.08,0.24 $\pm$ 0.04,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0
